## Setup and Installations

In [ ]:
pip install openai
!pip install --upgrade --force-reinstall pydantic pydantic-core
!pip install jiter

In [ ]:
import sqlite3
import openai
from openai import OpenAI
import re
import jiter

## Helper Functions

In [ ]:
def get_table_columns(table_name, db_path="travel2.sqlite"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    conn.close()
    return [col[1] for col in columns]

In [ ]:
def execute_sql(sql_query, db_path="travel2.sqlite", preview_limit=10):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        conn.close()
        return result[:preview_limit], len(result)
    except Exception as e:
        conn.close()
        return f" Error executing query: {e}", 0

In [ ]:
def show_tables(db_path="travel2.sqlite"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    print(" Tables in DB:", [table[0] for table in tables])

In [ ]:
def load_city_to_code_map(db_path="travel2.sqlite"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT city, airport_code FROM airports_data;")
    data = cursor.fetchall()
    conn.close()
    return {city.lower(): code for city, code in data}

In [ ]:
client = OpenAI(api_key="YOUR_API_KEY_HERE")
city_code_map = load_city_to_code_map()

def nl_to_sql(nl_query, table_name):
    # Try to handle simple "from <city>" case first with mapping
    user_input_lower = nl_query.lower()
    if "from" in user_input_lower:
        after_from = user_input_lower.split("from")[-1].strip()
        city_name = after_from.split()[0]  # can be improved later for multi-word cities
        airport_code = city_code_map.get(city_name)
        if airport_code:
            return f"SELECT * FROM flights WHERE departure_airport = '{airport_code}';"
    
    # fallback: use GPT to generate SQL
    columns = get_table_columns(table_name)
    prompt = (
        f"You are an expert SQL generator. The table '{table_name}' has columns: {columns}. "
        f"Translate the following natural language request into a SQL query only. "
        f"Do NOT include any explanation or markdown formatting like ```sql. Only output pure SQL."
        f"\nRequest: \"{nl_query}\""
    )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Translate natural language to SQL ONLY. Return only valid SQL without any explanation."},
            {"role": "user", "content": prompt}
        ]
    )

    raw_output = response.choices[0].message.content.strip()
    clean_sql = re.sub(r"```sql|```", "", raw_output).strip()
    return clean_sql

In [ ]:
def chatbot():
    print(" Natural Language to SQL Chatbot (type 'exit' to leave)")
    show_tables()
    
    while True:
        table_name = input("Which table do you want to query? (flights/airports_data/hotels): ").strip()
        user_input = input("\n Your request: ")
        if user_input.lower() == "exit":
            print(" Ended")
            break
        

        sql_query = nl_to_sql(user_input, table_name)
        print(f" GPT's SQL:\n{sql_query}")
        
        result, total = execute_sql(sql_query)
        if isinstance(result, str):  # error
            print(result)
        else:
            print(f" Showing first {len(result)} of {total} results:")
            for row in result:
                print(row)

## Run Chatbot

In [ ]:
chatbot()